In [ ]:
import boto3
import sagemaker


region = boto3.Session().region_name
role = sagemaker.get_execution_role()

pipeline_name = f"GLAIR-BCAF-Consultation-Training" # Change these to reflect your project/business name

In [ ]:
from pipelines.CarPriceML_Training.pipeline import get_pipeline


pipeline = get_pipeline(
    region=region,
    role=role,
    pipeline_name=pipeline_name
)

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
s3_singapore = boto3.client("s3", region_name="ap-southeast-1")

def get_latest_file(bucket_name, prefix_name):
    s3_uri_response = s3_singapore.list_objects_v2(Bucket=bucket_name, Prefix=prefix_name)
    latest_key = sorted(s3_uri_response.get("Contents", []), key=lambda x: x["LastModified"], reverse=True)[0]["Key"]

    return f"s3://{bucket_name}/{latest_key}"
    
s3_uri_lelang = get_latest_file(
    "glair-exploration-sagemaker-s3-bucket-singapore",
    "glair-bcaf-consultation-input/training/lelang"
)

s3_uri_crawling = get_latest_file(
    "glair-exploration-sagemaker-s3-bucket-singapore",
    "glair-bcaf-consultation-input/training/crawling"
)

In [ ]:
sagemaker_virginia = boto3.client("sagemaker", region_name="us-east-1")

def get_best_hyperparameter():
    hpo_response = sagemaker_virginia.list_hyper_parameter_tuning_jobs(
        SortBy='CreationTime',
        SortOrder='Descending',
        NameContains='Car',
        StatusEquals='Completed'
    )
    
    tuned_hyperparameter = sagemaker_virginia.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=hpo_response['HyperParameterTuningJobSummaries'][0]['HyperParameterTuningJobName']
    )
    
    return tuned_hyperparameter['BestTrainingJob']['TunedHyperParameters']

hyperparameters = get_best_hyperparameter()

In [ ]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.m5.xlarge",
        ProcessingInstanceCount="1",
        TrainingInstanceType="ml.m5.xlarge",
        TrainingInstanceCount="1",
        InputDataLelangURI=s3_uri_lelang,
        InputDataCrawlingURI=s3_uri_crawling,
        MaxDepth=hyperparameters['max_depth'],
        SubSample=hyperparameters['subsample'],
        ColSampleByTree=hyperparameters['colsample_bytree'],
        NumRound=hyperparameters['num_round'],
        ETA=hyperparameters['eta'],
        MinChildWeight=hyperparameters['min_child_weight'],
        Gamma=hyperparameters['gamma']
    )
)

In [ ]:
execution.describe()

In [7]:
MODEL_TYPES = ['toyota', 'daihatsu']
dict_model_types = {model_type: f'GLAIR-BCAF-Consultation-Training-{model_type.capitalize()}' for model_type in MODEL_TYPES}

In [10]:
model_type = 'toyota'
dict_model_types[model_type]

'GLAIR-BCAF-Consultation-Training-Toyota'

In [5]:
dict_model_types = {
    'key1': 'value1',
    'key2': 'value2',
    'some_key': 'GLAIR-BCAF-Consultation-Training-some_key',
}

value = dict_model_types['some_key']
value

'GLAIR-BCAF-Consultation-Training-some_key'